## Import data from csv using PySpark

In [38]:
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, from_unixtime

In [4]:
sc = SparkContext("local", "Simple App")

In [13]:
sqlContext = SQLContext(sc)

In [ ]:
# sc.stop()

In [5]:
spark = SparkSession.builder.master("local").\
appName("Simple App").\
config("spark.some.config.option", "some-value").\
getOrCreate()

In [6]:
sample_1_path = 'xian_sample_1.csv'
geo_1_path = 'gps_20161001.csv'
sample_dir = 'data/'

### Version 1: Import RDD from csv, followed by DF creation

In [7]:
sample_rdd = sc.textFile(sample_dir + sample_1_path)

In [14]:
didi_data_1 = sample_rdd.map(lambda l: l.split(','))
didi_data_2 = didi_data_1.map(lambda row: (
row[0], row[1], row[2], float(row[3]), float(row[4])))

In [9]:
schemaString = "driver_id order_id timestamp lon lat"

In [10]:
fields = [StructField(field_name, StringType(), True) for field_name in schemaString.split()]
schema = StructType(fields)

In [15]:
# didi_df = spark.createDataFrame(didi_data, schema)
didi_df = spark.createDataFrame(didi_data_2, schema)
print(type(didi_df))

<class 'pyspark.sql.dataframe.DataFrame'>


In [16]:
didi_df.printSchema()
didi_df.head(5)

root
 |-- driver_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- lon: string (nullable = true)



[Row(driver_id='a01b8439e1e42ffcd286241b04d9b1b5', order_id='f11440a64a0f084fe346a398c62aa9ad', timestamp='1475277482', lat='108.92466', lon='34.27657'),
 Row(driver_id='a01b8439e1e42ffcd286241b04d9b1b5', order_id='f11440a64a0f084fe346a398c62aa9ad', timestamp='1475277488', lat='108.92527', lon='34.27658'),
 Row(driver_id='a01b8439e1e42ffcd286241b04d9b1b5', order_id='f11440a64a0f084fe346a398c62aa9ad', timestamp='1475277506', lat='108.9276', lon='34.27659'),
 Row(driver_id='a01b8439e1e42ffcd286241b04d9b1b5', order_id='f11440a64a0f084fe346a398c62aa9ad', timestamp='1475277476', lat='108.92399', lon='34.27655'),
 Row(driver_id='a01b8439e1e42ffcd286241b04d9b1b5', order_id='f11440a64a0f084fe346a398c62aa9ad', timestamp='1475277515', lat='108.9291', lon='34.2766')]

### Version 2: Direct DF import from csv

In [95]:
df = sqlContext.read.csv(sample_dir + sample_1_path, )

In [97]:
df.printSchema()
df.head(5)

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)



[Row(_c0='a01b8439e1e42ffcd286241b04d9b1b5', _c1='f11440a64a0f084fe346a398c62aa9ad', _c2='1475277482', _c3='108.92466', _c4='34.27657'),
 Row(_c0='a01b8439e1e42ffcd286241b04d9b1b5', _c1='f11440a64a0f084fe346a398c62aa9ad', _c2='1475277488', _c3='108.92527', _c4='34.27658'),
 Row(_c0='a01b8439e1e42ffcd286241b04d9b1b5', _c1='f11440a64a0f084fe346a398c62aa9ad', _c2='1475277506', _c3='108.9276', _c4='34.27659'),
 Row(_c0='a01b8439e1e42ffcd286241b04d9b1b5', _c1='f11440a64a0f084fe346a398c62aa9ad', _c2='1475277476', _c3='108.92399', _c4='34.27655'),
 Row(_c0='a01b8439e1e42ffcd286241b04d9b1b5', _c1='f11440a64a0f084fe346a398c62aa9ad', _c2='1475277515', _c3='108.9291', _c4='34.2766')]

## Data Processing

In [40]:
new_didi_df = didi_df.withColumn('timestamp_new', from_unixtime("timestamp"))

In [42]:
print(new_didi_df.head())

Row(driver_id='a01b8439e1e42ffcd286241b04d9b1b5', order_id='f11440a64a0f084fe346a398c62aa9ad', timestamp='1475277482', lat='108.92466', lon='34.27657', timestamp_new='2016-10-01 07:18:02')


In [46]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank, dense_rank
window = Window.partitionBy(new_didi_df['order_id']).orderBy(new_didi_df['timestamp_new'])

In [49]:
new_didi_df.select('*', dense_rank().over(window).alias('index')).show()

+--------------------+--------------------+----------+---------+--------+-------------------+-----+
|           driver_id|            order_id| timestamp|      lat|     lon|      timestamp_new|index|
+--------------------+--------------------+----------+---------+--------+-------------------+-----+
|a01b8439e1e42ffcd...|210acc529e4b9fc55...|1475329529|108.94276|34.27099|2016-10-01 21:45:29|    1|
|a01b8439e1e42ffcd...|210acc529e4b9fc55...|1475329532|108.94276|34.27099|2016-10-01 21:45:32|    2|
|a01b8439e1e42ffcd...|210acc529e4b9fc55...|1475329535|108.94262|34.27108|2016-10-01 21:45:35|    3|
|a01b8439e1e42ffcd...|210acc529e4b9fc55...|1475329538|108.94242|34.27108|2016-10-01 21:45:38|    4|
|a01b8439e1e42ffcd...|210acc529e4b9fc55...|1475329541|108.94225|34.27108|2016-10-01 21:45:41|    5|
|a01b8439e1e42ffcd...|210acc529e4b9fc55...|1475329544|108.94204|34.27108|2016-10-01 21:45:44|    6|
|a01b8439e1e42ffcd...|210acc529e4b9fc55...|1475329547|108.94186|34.27108|2016-10-01 21:45:47|    7|


In [61]:
ranked_df = new_didi_df.withColumn('rank', 
                                  dense_rank().over(window))
ranked_df.show(5)

+--------------------+--------------------+----------+---------+--------+-------------------+----+
|           driver_id|            order_id| timestamp|      lat|     lon|      timestamp_new|rank|
+--------------------+--------------------+----------+---------+--------+-------------------+----+
|a01b8439e1e42ffcd...|210acc529e4b9fc55...|1475329529|108.94276|34.27099|2016-10-01 21:45:29|   1|
|a01b8439e1e42ffcd...|210acc529e4b9fc55...|1475329532|108.94276|34.27099|2016-10-01 21:45:32|   2|
|a01b8439e1e42ffcd...|210acc529e4b9fc55...|1475329535|108.94262|34.27108|2016-10-01 21:45:35|   3|
|a01b8439e1e42ffcd...|210acc529e4b9fc55...|1475329538|108.94242|34.27108|2016-10-01 21:45:38|   4|
|a01b8439e1e42ffcd...|210acc529e4b9fc55...|1475329541|108.94225|34.27108|2016-10-01 21:45:41|   5|
+--------------------+--------------------+----------+---------+--------+-------------------+----+
only showing top 5 rows



In [68]:
joined_df = ranked_df.alias('a').join(ranked_df.alias('b'),
                                      on = 'order_id').\
where('a.rank == b.rank-1').show(5)

+--------------------+--------------------+----------+---------+--------+-------------------+----+--------------------+----------+---------+--------+-------------------+----+
|            order_id|           driver_id| timestamp|      lat|     lon|      timestamp_new|rank|           driver_id| timestamp|      lat|     lon|      timestamp_new|rank|
+--------------------+--------------------+----------+---------+--------+-------------------+----+--------------------+----------+---------+--------+-------------------+----+
|8c1490989064df3c0...|a01b8439e1e42ffcd...|1475334773| 108.9874|34.21696|2016-10-01 23:12:53|   9|a01b8439e1e42ffcd...|1475334776|108.98698| 34.2172|2016-10-01 23:12:56|  10|
|f11440a64a0f084fe...|a01b8439e1e42ffcd...|1475277728|108.95916|34.27597|2016-10-01 07:22:08|  68|a01b8439e1e42ffcd...|1475277731|108.95928|34.27598|2016-10-01 07:22:11|  69|
|f11440a64a0f084fe...|a01b8439e1e42ffcd...|1475277605|108.94605|34.27678|2016-10-01 07:20:05|  29|a01b8439e1e42ffcd...|147527

In [72]:
joined_df = ranked_df.alias('a').join(ranked_df.alias('b'),
                                      $'a.order_id'==$'b.order_id').\
where('a.rank == b.rank-1').show(5)

SyntaxError: invalid syntax (<ipython-input-72-95cc93c850d0>, line 2)